In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm.notebook import tqdm
import datetime as dt
import matplotlib.pyplot as plt
import datetime
import time
import math

In [2]:
train_2017 = pd.read_csv(r'../data/train_2017.csv',low_memory=False)
train_2018 = pd.read_csv(r'../data/train_2018.csv',low_memory=False)
test_2019 = pd.read_csv(r'../data/test_2019.csv',low_memory=False)
test_2020 = pd.read_csv(r'../data/test_2020.csv',low_memory=False)

#读取统计数据
count_2017=pd.read_csv(r'../data/count_2017.csv')
count_2018=pd.read_csv(r'../data/count_2018.csv')
count_2019=pd.read_csv(r'../data/count_2019.csv')
count_2020=pd.read_csv(r'../data/count_2020.csv')

In [3]:
#整理出训练集测试集时间列表
lst_1718 ,lst_1920 = [],[]
lst_1718 = list(count_2017['date'].unique())
lst_1718.extend(list(count_2018['date'].unique()))
 
lst_1920 = list(count_2019['date'].unique())
lst_1920.extend(list(count_2020['date'].unique()))

#生成病例名称和统计函数的全局列表变量
nameLst = count_2017['admin_illness_name'].unique()
funLst = ['mean','sum','median','var']

def prepare(data,timeLst):
    #从大表中提取计数和医疗费用特征
    df = data
    df.drop(['org_code','district_code','treatment_section_code','admin_illness_code'],axis=1,inplace=True)
    df = df[df['admin_illness_name'].isin(nameLst)].sort_values(by=['admin_illness_name','treatment_date']).reset_index(drop=True)
    df1 = df.groupby(['admin_illness_name','treatment_date'])['diagnosis_all_cost'].agg(funLst)
    #df2 = df.groupby(['admin_illness_name','treatment_date'])['diagnosis_all_cost'].quantile(0.9)
    #df3 = df.groupby(['admin_illness_name','treatment_date'])['diagnosis_all_cost'].quantile(0.1)
    #df2.name,df3.name = 'quantile0.9','quantile0.1'
    #Temp =  pd.concat([df1,df2,df3],axis=1).reset_index()
    temp = df.groupby(['admin_illness_name','treatment_date']).count().reset_index().iloc[:,:3].rename(columns={'falg_first_visit':'count'})
    data = pd.merge(left=temp,right=df1.reset_index()).rename(columns={'treatment_date':'date'})
    
    #缺失值处理,注意现在缺失值是count补0，其他向上补齐，若效果不好，一会试试都补0
    Indexdf = pd.DataFrame({'date':timeLst})
    data_df = pd.DataFrame()
    for name in nameLst:
        tar = data[data['admin_illness_name']==name]
        inner = pd.merge(tar,Indexdf,how='outer',left_index=False,right_index=True,on='date' )
        #inner['count'].interpolate(method='linear', limit_direction='forward', axis=0)
        inner['count'].fillna(value=0,axis=0,inplace=True)
        inner.fillna(method='ffill',axis=0,inplace=True)
        data_df = pd.concat([data_df,inner])
    data_df = data_df.sort_values(by=['admin_illness_name','date']).reset_index(drop=True)
    data_df[['date','count']] = data_df[['date','count']].astype(int)
    data = data_df
    
    #日期处理
    data['time'] = data.date.apply(lambda x:str(x))
    data['time'] = pd.to_datetime(data['time'],format='%Y-%m-%d')
    data['year_id'] = data['time'].apply(lambda x : x.year)
    data['wkday_id'] = data['time'].apply(lambda x: x.weekday())
    data['month_id'] = data['time'].apply(lambda x : x.month)
    name_label = dict(zip(sorted(list(set(data['admin_illness_name']))), range(0, len(set(data['admin_illness_name'])))))
    data['name_label'] = data['admin_illness_name'].map(name_label)
    data.drop(['time'],axis=1,inplace=True)
    data['count'] = data['count'].apply(lambda x: x if x==0 else math.log(x+1,2))
    for i in tqdm(range(1,8)):
        data['count'+'_shifts_'+str(i)]=data.groupby(['admin_illness_name'])['count'].shift(i)
        
    data['diff_1']=data['count']-data['count_shifts_1']
    data['1_3_max'] = data.loc[:,'count_shifts_1':'count_shifts_3'].max(1)
    data['1_3_min'] = data.loc[:,'count_shifts_1':'count_shifts_3'].min(1)
    data['1_3_mean'] = data.loc[:,'count_shifts_1':'count_shifts_3'].mean(1)
    return data


In [4]:
def prepare2(data):
    
    fea = pd.read_csv(r'../feature/weather.csv')
    fea=fea[['平均气温 ','最低气温','最高气温','最低气温_7_min','最低气温_7_var','日期']]
    temp_list=['最低气温']
    #fea['最低气温'+'_afters_'+str(-7)]=fea['最低气温'].shift(-7)
    fea['最低气温_7_var'+'_afters_'+str(-7)]=fea['最低气温_7_var'].shift(-7)
    fea['最低气温_7_min'+'_afters_'+str(-7)]=fea['最低气温_7_min'].shift(-7)
    fea['min_7_distance']=fea['最低气温_7_min_afters_-7']-fea['最低气温_7_min']

    temp_columns=fea.columns
    temp_columns=list(temp_columns)
    new_columns=[]
    for i in temp_columns:
        i=i.replace('气温','temp')
        i=i.replace('平均','avg')
        i=i.replace('最高','max')
        i=i.replace('最低','min')
        i=i.replace('温度差','temp_distance')
        new_columns.append(i)
    fea.columns=new_columns
    fea = fea.iloc[:,1:]
    
    data = pd.merge(data,fea,how='inner',left_on='date',right_on='日期')
    data.drop(['日期'],axis=1,inplace=True)
    
    return data

In [5]:
def prepare3(data):
    pneumonia=data[data['admin_illness_name']=='肺炎'][['date','count']]
    cough=data[data['admin_illness_name']=='咳嗽'][['date','count']]
    herpetic_pharyngitis=data[data['admin_illness_name']=='疱疹性咽峡炎'][['date','count']]
 
    pneumonia.columns=['date','pneumonia_count']
    cough.columns=['date','cough_count']
    herpetic_pharyngitis.columns=['date','herpetic_pharyngitis_count']
 
    data = pd.merge(data,pneumonia,how='left',on='date')
    data = pd.merge(data,cough,how='left',on='date')
    data = pd.merge(data,herpetic_pharyngitis,how='left',on='date')
  
    
    lst1 = ['上呼吸道感染','呼吸道感染','头晕','头痛','急性扁桃体炎','消化不良','皮炎','结膜炎','腹痛']
    lst2 = ['上呼吸道疾病','丘疹性荨麻疹','中耳炎','发热','变应性鼻炎','呕吐','咳嗽','哮喘','急性上呼吸道感染','急性喉炎','手足口病','支气管炎']
    lst22 = ['湿疹','猩红热样红斑', '疱疹性咽峡炎', '皮疹','肺炎','胃肠功能紊乱','腹泻','荨麻疹']
    lst3 = ['外伤证']
    yiyuan = dict(zip(lst1+lst2+lst22+lst3,[0 for x in lst1]+[1 for y in lst2+lst22]+[2 for z in lst3]))
    data['yiyuan'] = data['admin_illness_name'].map(yiyuan)
    
    return data

In [6]:
df_train = pd.concat([train_2017,train_2018]).iloc[:,4:]
df_test  = pd.concat([test_2019,test_2020]).iloc[:,4:]
count_train = prepare(df_train,lst_1718)
count_test  = prepare(df_test,lst_1920) 
count_train = prepare2(count_train)
count_test  = prepare2(count_test)
count_train = prepare3(count_train)
count_test  = prepare3(count_test)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
def get_train(df):
    df_train = pd.DataFrame()
    data = df
    for id_ in data.admin_illness_name.unique():
        tmp = data[data.admin_illness_name == id_].reset_index(drop=True)
        tmp['count_1'] = tmp['count'].shift(-1)
        tmp['count_2'] = tmp['count'].shift(-2)
        tmp['count_3'] = tmp['count'].shift(-3)
        tmp['count_4'] = tmp['count'].shift(-4)
        tmp['count_5'] = tmp['count'].shift(-5)
        tmp['count_6'] = tmp['count'].shift(-6)
        tmp['count_7'] = tmp['count'].shift(-7)
        tmp['count_8'] = tmp['count'].shift(-8)
        tmp['count_9'] = tmp['count'].shift(-9)
        tmp['count_10'] = tmp['count'].shift(-10)
        tmp['count_11'] = tmp['count'].shift(-11)
        tmp['count_12'] = tmp['count'].shift(-12)
        tmp['count_13'] = tmp['count'].shift(-13)
        tmp['count_14'] = tmp['count'].shift(-14)
        df_train = df_train.append(tmp)
        
    return df_train

In [8]:
df_train = get_train(count_train)

#加入2019年的数据并最后与df_train合并
dateLst = test_2019.treatment_date.unique()
df_month2  = test_2019[test_2019.treatment_date.isin(dateLst[:28])]
df_month5  = test_2019[test_2019.treatment_date.isin(dateLst[28:28+31])]
df_month8  = test_2019[test_2019.treatment_date.isin(dateLst[28+31:28+31+31])]
df_month11 = test_2019[test_2019.treatment_date.isin(dateLst[28+31+31:28+31+31+31])]
df_month = [df_month2,df_month5,df_month8,df_month11]
lst_time = [dateLst[:28],dateLst[28:28+31],dateLst[28+31:28+31+31],dateLst[28+31+31:28+31+31+31]]

for i in range(len(df_month)):
    df_piece = prepare(df_month[i].iloc[:,4:],lst_time[i])
    df_piece = prepare2(df_piece)
    df_piece = prepare3(df_piece)
    df_piece = get_train(df_piece)
    df_train = pd.concat([df_train,df_piece])
    
#加入2020年的数据并最后与df_train合并
dateLst   = test_2020.treatment_date.unique()
df_month3 = test_2020[test_2020.treatment_date.isin(dateLst[:31])]
df_month6 = test_2020[test_2020.treatment_date.isin(dateLst[31:62])]
df_month = [df_month3,df_month6]
lst_time = [dateLst[:31],dateLst[31:62]]
for i in range(len(df_month)):
    df_piece = prepare(df_month[i].iloc[:,4:],lst_time[i])
    df_piece = prepare2(df_piece)
    df_piece = prepare3(df_piece)
    df_piece = get_train(df_piece)
    df_train = pd.concat([df_train,df_piece])

df_train = df_train.sort_values(by=['admin_illness_name','date']).reset_index(drop=True)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
count_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21900 entries, 0 to 21899
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   admin_illness_name          21900 non-null  object 
 1   date                        21900 non-null  int32  
 2   count                       21900 non-null  float64
 3   mean                        21900 non-null  float64
 4   sum                         21900 non-null  float64
 5   median                      21900 non-null  float64
 6   var                         21859 non-null  float64
 7   year_id                     21900 non-null  int64  
 8   wkday_id                    21900 non-null  int64  
 9   month_id                    21900 non-null  int64  
 10  name_label                  21900 non-null  int64  
 11  count_shifts_1              21870 non-null  float64
 12  count_shifts_2              21840 non-null  float64
 13  count_shifts_3              218

In [10]:
def get_test(df):
    d = df.copy().reset_index(drop=True)
    d1 = d[d["date"]==20190228].reset_index(drop=True)
    d2 = d[d["date"]==20190531].reset_index(drop=True)
    d3 = d[d["date"]==20190831].reset_index(drop=True)
    d4 = d[d["date"]==20191130].reset_index(drop=True)
    d5 = d[d["date"]==20200331].reset_index(drop=True)
    d6 = d[d["date"]==20200630].reset_index(drop=True)
    return pd.concat([d1,d2,d3,d4,d5,d6],axis=0).reset_index(drop=True)

In [11]:
target_cols = ['count_'+str(i) for i in range(1,15)]
feats_cols = [col for col in df_train.columns if col not in target_cols+["admin_illness_name","date",'year_id']]
cate_feat = ['name_label','wkday_id','month_id','yiyuan']
print(target_cols)
print(feats_cols)
print(cate_feat)

['count_1', 'count_2', 'count_3', 'count_4', 'count_5', 'count_6', 'count_7', 'count_8', 'count_9', 'count_10', 'count_11', 'count_12', 'count_13', 'count_14']
['count', 'mean', 'sum', 'median', 'var', 'wkday_id', 'month_id', 'name_label', 'count_shifts_1', 'count_shifts_2', 'count_shifts_3', 'count_shifts_4', 'count_shifts_5', 'count_shifts_6', 'count_shifts_7', 'diff_1', '1_3_max', '1_3_min', '1_3_mean', 'mintemp', 'maxtemp', 'mintemp_7_min', 'mintemp_7_var', 'mintemp_7_var_afters_-7', 'mintemp_7_min_afters_-7', 'min_7_distance', 'pneumonia_count', 'cough_count', 'herpetic_pharyngitis_count', 'yiyuan']
['name_label', 'wkday_id', 'month_id', 'yiyuan']


In [12]:
def get_model_type():   
    model = lgb.LGBMRegressor(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
            max_depth=7, learning_rate=0.05, min_child_samples=5, random_state=2021,
            n_estimators=5000, subsample=0.9, colsample_bytree=0.7,
            )
    return model

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
def run_lgb(df_train, df_test,target,cate_feat,rank_Seires,feats_cols):
    model = get_model_type()
    oof = np.zeros(df_train.shape[0])
    preds = np.zeros(df_test.shape[0])
    test_score = []
    train_score = []
    kfold = KFold(n_splits=5,shuffle=True, random_state=1212)
    for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train,target)):
        
        X_train = df_train.iloc[trn_idx]
        Y_train = target.iloc[trn_idx]
        X_val = df_train.iloc[val_idx]
        Y_val = target.iloc[val_idx]
        lgb_model = model.fit(X_train, 
                              Y_train,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=0,
                              eval_metric='mse',
                              early_stopping_rounds=50,
                              categorical_feature=cate_feat)
        oof[val_idx] = (2**lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration_))-1
        preds += ((2**lgb_model.predict(df_test, num_iteration=lgb_model.best_iteration_))-1)/5
        y_train_predict = lgb_model.predict(X_train, num_iteration=lgb_model.best_iteration_)
        y_test_predict = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration_)
        train_mse = mean_squared_error(2**Y_train-1,2**y_train_predict-1)
        test_mse = mean_squared_error(2**Y_val-1,2**y_test_predict-1)
        print('----训练MSE为：',train_mse,'-----')
        print('----测试MSE为：',test_mse,'-----')
        test_score.append(test_mse)
        train_score.append(train_mse)

    print('训练集平均误差为:',np.mean(train_score))
    print('测试集平均误差为:',np.mean(test_score),'\n')
    
    ans = pd.Series(lgb_model.feature_importances_,index=feats_cols)
    rank_Seires += ans.rank(method='first')
    score = mean_squared_error(oof, 2**target-1) 
    
    return oof, preds,score,rank_Seires,ans

In [13]:
import warnings
warnings.filterwarnings('ignore')
scores1=[]
sub=pd.DataFrame()
rank_Seires = pd.Series(np.zeros(len(feats_cols)),index=feats_cols)

i=0
scores2=[]
count_test=count_test.dropna().reset_index(drop=True)
test_x = get_test(count_test)
t_id   = test_x[feats_cols]


for col in target_cols:
    i += 1
    if i <= 13:
        df_temp = df_train.iloc[:,:-14+i]
        df_temp = df_temp.dropna().reset_index(drop=True)
    else:
        df_temp = df_train.iloc[:,:]
        df_temp = df_temp.dropna().reset_index(drop=True)
    print(df_temp.shape)
    x_id = df_temp[feats_cols]
    y_id = df_temp[col]
    print('未来第',i,'天')
    oof, preds, score,rank_Seires,ans=run_lgb(x_id, t_id, y_id,cate_feat,rank_Seires,feats_cols)
    test_x[col]=preds
    scores1.append(score)

sub = pd.concat([sub,test_x],axis=0)
print(rank_Seires.sort_values(ascending=False))
print(x_id.shape,t_id.shape,y_id.shape)
print('14天总测试集误差为',np.mean(scores1))

(25543, 34)
未来第 1 天
----训练MSE为： 87.62572298847792 -----
----测试MSE为： 125.41568891382782 -----
----训练MSE为： 98.01106602859404 -----
----测试MSE为： 117.21586512498337 -----
----训练MSE为： 101.77633094624142 -----
----测试MSE为： 115.37912514158184 -----
----训练MSE为： 95.51623549448654 -----
----测试MSE为： 124.32312301868595 -----
----训练MSE为： 95.1509566938753 -----
----测试MSE为： 128.56863742089635 -----
训练集平均误差为: 95.61606243033505
测试集平均误差为: 122.18048792399506 

(25337, 35)
未来第 2 天
----训练MSE为： 102.70364736018392 -----
----测试MSE为： 141.89408134983046 -----
----训练MSE为： 114.67677256976673 -----
----测试MSE为： 129.45919191454726 -----
----训练MSE为： 101.64914318240515 -----
----测试MSE为： 148.0533513697884 -----
----训练MSE为： 107.19484834415876 -----
----测试MSE为： 177.49258338195884 -----
----训练MSE为： 137.63757997572483 -----
----测试MSE为： 159.76768642007872 -----
训练集平均误差为: 112.77239828644788
测试集平均误差为: 151.3333788872407 

(25130, 36)
未来第 3 天
----训练MSE为： 101.9522842687569 -----
----测试MSE为： 140.20055331250165 -----
----训练MSE为： 119

In [14]:
sub.to_csv("../user_data/lgb1_result.csv",index=False)

In [15]:
#预测目标是未来十四（*6）天的三十种病例的数量
# to_predict=pd.read_csv(r'../data/to_predict.csv')
# to_predict["count"]=0

# for c in sub.admin_illness_name.unique():
#     a=sub.loc[sub['admin_illness_name']==c,target_cols].values.flatten()
#     to_predict.loc[to_predict['admin_illness_name']==c,"count"]=a

In [16]:
# timenow =time.strftime("%Y-%m-%d",time.localtime()) 
# # times = 2
# #to_predict['count'] = round(to_predict['count'])
# to_predict['count']=to_predict['count'].astype(int)
# to_predict[["id","count"]].to_csv("../user_data/lgb1_result.csv",index=False)
# to_predict.to_excel("F:/山东数据/山东数据大赛/健康医疗/预测结果/to_predict_{}_{}_{}.xls".format(round(np.mean(scores1),6),timenow,times),index=False)
# to_predict[["id","count"]].to_csv("F:/山东数据/山东数据大赛/健康医疗/预测结果/submit_{}_{}_{}.csv".format(round(np.mean(scores1),6),timenow,times),index=False)
# to_predict.to_excel("F:/山东数据/山东数据大赛/健康医疗/预测结果/to_predict_{}_{}_{}.xls".format(round(np.mean(scores1),6),timenow,times),index=False)